# Ausgangslage: E-Books in Deutschland

## Passende Daten suchen
- [Bibliotheksstatistik](https://www.bibliotheksstatistik.de/vaZwischen) mit Daten über aktive Nutzer*innen und Ausleihen (**auswertung.csv**)
- [Bevölkerungsstatistik](https://www-genesis.destatis.de/genesis//online?operation=table&code=12411-0001&bypass=true&levelindex=1&levelid=1635785462810#abreadcrumb) mit Bevölerungsstand und -fortschreibung (**bevoelkerung.csv**)
- [Statista](https://de.statista.com/statistik/studie/id/6689/dokument/e-books-statista-dossier/) (S. 5) mit Daten über die Anzahl von Menschen, die E-Books lesen
- [Statista](https://de.statista.com/statistik/daten/studie/232191/umfrage/absatz-von-e-books-in-deutschland/) mit Daten über den Absatz von E-Books
- [Statista](https://de.statista.com/statistik/daten/studie/171899/umfrage/haeufigkeit-buecher-lesen-in-der-freizeit/) mit Daten über die Lesegewohnheiten der Deutschen



In [25]:
# import modules
import pandas as pd

In [26]:
# population
population_df = pd.read_csv('bevoelkerung.csv', delimiter=';', header=0, encoding='UTF-8')
population_df = population_df.rename(columns={'Bevoelkerungsstand': 'year', 'Anzahl': 'population'})
population_df['year'] = pd.to_datetime(population_df['year'], format='%d.%m.%Y')

In [27]:
# library
library_df = pd.read_csv('auswertung.csv', delimiter=';', header=[0], encoding='ISO-8859-1')

library_df = pd.DataFrame({
    'year' : ([2017, 2018, 2019, 2020]),
    'lenders' : ([library_df['Entleiher2017'][0], library_df['Entleiher2018'][0], library_df['Entleiher2019'][0], library_df['Entleiher2020'][0]]),
    'lendings' : ([library_df['Entleih. insges.2017'][0], library_df['Entleih. insges.2018'][0], library_df['Entleih. insges.2019'][0], library_df['Entleih. insges.2020'][0]]),
    'digital_lendings' : ([library_df['Entl. virt.Best.2017'][0], library_df['Entl. virt.Best.2018'][0], library_df['Entl. virt.Best.2019'][0], library_df['Entl. virt.Best.2020'][0]]),
})

## Daten in einem DataFrame zusammenfügen
Im DataFrame sind folgende Angaben im Zeitraum 2017-2020:
- der Bevölkerungsstand
- Anzahl der Menschen, die E-Books lesen
- Anzahl der Menschen, die mehrmals die Woche lesen
- Anzahl der Menschen, die mehrmals im Monat lesen
- Anzahl der Menschen, die einmal im Monat lesen
- Anzahl der verkauften E-Book
- Anzahl der aktiven Bücherei-Nutzer*innen
- Anzahl der ausgeliehen Medien
- Anzahl der ausgeliehen digitalen Medien


In [28]:
df = pd.DataFrame(
    {
        # general information
        'year': population_df['year'],
        'population': population_df['population'],

        # reading habits
        'digital_readers': ([7650000, 8520000, 8680000, 9120000]),
        'readers_weekly' : ([11640000, 11930000, 11960000, 12620000]),
        'readers_monthly' : ([15070000, 14840000, 14730000, 14710000]),
        'readers_once_month' : ([6870000, 7060000, 7110000, 7030000]),

        # publishing information
        'ebook_sales': ([29100000, 32800000, 32400000, 35800000]),
        
        # library information
        'lenders': library_df['lenders'],
        'lendings': library_df['lendings'],
        'digital_lendings': library_df['digital_lendings'], 
    }
)

### Neue Daten aggregieren
Zudem werten wir einige Daten aus:
- Anzahl der Menschen, die regelmäßig lesen (Summe der wöchentlichen, monatlichen und den Leser*innen, die nur einmal Monat lesen)
- Anteil der Menschen, die E-Books lesen
- Anteil der Menschen, die regelmäßig lesen

In [29]:
# regular readers
df['readers'] = df['readers_weekly'] + df['readers_monthly'] + df['readers_once_month']

# readers relative to the population over the years
df['perc_digital_readers'] = df['digital_readers'] / df['population']
df['perc_readers'] = df['readers'] / df['population']

## Daten visualisieren

In [30]:
import plotly.express as px
import plotly.graph_objects as go

### Ausleih und Verkaufszahlen von E-Books

Erkennbar ist, dass die Verkaufszahlen seit 2017 stagnieren, während die Ausleihe steigen. Diese haben 2019 erstmals die Verkaufszahlen überstiegen. Während dem seit der Covid-19-Pandemie sind keine besonderen Entwicklung zu bemerken.

In [31]:
fig = go.Figure()

# traces: lendings and sales
fig.add_trace(go.Scatter(x=df['year'], y=df['digital_lendings'], name='Ausleihe', line=dict(color='#CC9DA8', width=3)))
fig.add_trace(go.Scatter(x=df['year'], y=df['ebook_sales'], name='Verkäufe', line=dict(color='#810a26', width=3)))

fig.update_traces(hovertemplate='%{x}: %{y} E-Books')

# layout
fig.update_layout(title='Ausleih- und Verkaufszahlen von E-Books',
                    title_font_size=25,

                    xaxis_title='Jahr',
                    yaxis_title='Stückzahl',

                    font_family='Gravitas One',
                    font_color='#555555',

                    paper_bgcolor='#e2e2e2',
                    plot_bgcolor='#ffffff',
                   )

fig.update_xaxes(dtick="M12", tickformat="%Y", tick0='2017-12-31', title_text='Jahr', ticks='inside', tickcolor='#555555', fixedrange=True)
fig.update_yaxes(dtick=10000000, tick0=0, tickformat='.2s', ticks='inside', fixedrange=True, range=[0, 45000000], tickcolor='#555555')

# adding markers to indicate covid-19 and the point where lendings become higher than sales
fig.add_vline('2019-12-31', line=dict(color='#555555', width=2, dash='dash'))
fig.add_vrect(x0='2020-03-01', x1='2021-01-25', fillcolor='#e2e2e2', opacity=0.2, line=dict(color='#555555', width=1))

fig.show()

### Lesegewohnheiten der Deutschen
Erkennbar ist, dass sich die Lesegewohnheiten seit Jahren nicht verändern. Lediglich der Anteil der Menschen, die E-Books lesen, ist minimal gestiegen.

In [32]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df['year'], y=df['perc_readers'], name='lesen regelmäßig', line=dict(color='#CC9DA8', width=3)))
fig.add_trace(go.Scatter(x=df['year'], y=df['perc_digital_readers'], name='lesen E-Books', line=dict(color='#810a26', width=3)))

fig.add_vrect(x0='2020-03-01', x1='2021-01-25', fillcolor='#e2e2e2', opacity=0.2, line=dict(color='#555555', width=1))

fig.update_layout(title='Lesegewohnheiten der Deutschen',
                    title_font_size=25,

                    xaxis_title='Jahr',
                    yaxis_title='Anteil',

                    font_family='Gravitas One',
                    font_color='#555555',

                    paper_bgcolor='#e2e2e2',
                    plot_bgcolor='#ffffff',
                   )

fig.add_hline(df['perc_digital_readers'].iloc[-1], line=dict(color='#555555', width=1, dash='dash'))

fig.update_xaxes(dtick="M36", tickformat="%Y", tick0='2017-12-31', title_text='Jahr', ticks='inside', tickcolor='#555555', fixedrange=True)
fig.update_yaxes(dtick=0.1, tick0=0, tickformat="%", ticks='inside', fixedrange=True, range=[0, 0.45], tickcolor='#555555')

fig.show()